<a href="https://colab.research.google.com/github/patilshrinivas078/crewai/blob/main/CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


LangChain vs CrewAI </br>
Langchain: Individual agents that can use LLMs and tools </br>
CrewAI: Group of agents

Agents: Employee --> role, goal, backstory </br>
Task: --> description, expected_output, agent </br>
Tool: </br>
Crew

In [ ]:
from google.colab import userdata
import os
openai_key = userdata.get('openai_key')
os.environ['OPENAI_API_KEY'] = openai_key

In [ ]:
from langchain_openai import OpenAI

In [ ]:
llm = OpenAI()

In [ ]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of lancedb to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import crewai

In [ ]:
from crewai import Agent, Task, Crew

Agent Parameters: </br>
role</br>
goal</br>
backstory</br>
---------------</br>
llm </br>
tools </br>
max_iter  Default=25</br>
verbose </br>
allow_delegation Default=True

Task Parameters: </br>
description </br>
expected_output </br>
agent </br>
------------</br>
context </br>
output_json </br>
output_file </br>

In [ ]:
from langchain_community.tools import tool

@tool
def get_today_date(input=""):
  '''Get today's date'''
  from datetime import date
  return str(date.today())

In [ ]:
information_agent = Agent(
    role = "Information Agent",
    goal = "Give brief information about {topic}",
    backstory = "You are intelligent and helpful assistant; you love to obtain information and share knowledge with others",
    llm=llm,
    # tools = [tool]
    tools = [get_today_date] #using tool decorator
)

In [ ]:
task = Task(
    description = "Create a paragraph to explain {topic}",
    expected_output = "quick summary of the topic with 5 bullet points",
    agent = information_agent
)

In [ ]:
crew = Crew(
    agents = [information_agent],
    tasks = [task],
    verbose=True
)

In [ ]:
result = crew.kickoff(inputs={'topic':"Data Science"})

# Agent: Information Agent
## Task: Create a paragraph to explain Data Science


# Agent: Information Agent
## Using tool: get_today_date
## Tool Input: 
"{\"input\": {\"description\": \"Today's date\", \"type\": \"str\"}}"
## Tool Output: 
2024-12-29


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: get_today_date
Tool Arguments: {'input': {'description': None, 'type': 'Any'}}
Tool Description: Get today's date

Use the following format:

Thought: you should always think about what to do
Action: the action to take, only one name of [get_today_date], just the name, exactly as it's written.
Action Input: the input to the action, just a simple python dictionary, enclosed in curly braces, using " to wrap keys and values.
Observation: the result of the action

Once all necessary information is gathered:

Thought: I now know the final answer
Final Answer: the final answer to the original input question



# Agent: Information

In [ ]:
result.raw

'The final answer to the original input question is that Data Science is a multidisciplinary field that combines computer science, statistics, and domain expertise to obtain valuable insights and information from data. It involves using various tools and techniques, such as statistical analysis, machine learning, and data visualization, to extract knowledge and make informed decisions. It is a rapidly growing field with applications in various industries, including business, healthcare, and government.'

# Blog Creation Crew

Planner: Outline </br>
Writer: Writing </br>
Editor: Checks grammar, safe information

### Agents

In [ ]:
planner = Agent(
    role = "Content Planner",
    goal = "Create a blog post outline on {topic}",
    backstory = '''Your job is to plan a blog post. You excel at
    outlining the blogs on given topics so that it can be further used for writing.''',
    llm = llm
)

writer = Agent(
    role = "Content Writer",
    goal = "Write a creative and engaging content for a blog on {topic}",
    backstory = '''You are very good at storytelling and creative writing about any topic''',
    llm = llm
)

editor = Agent(
    role = "Content Editor",
    goal = "To edit a given blog post",
    backstory = '''Your job is to review to check for any grammatical error
    and ensure that it contains safe information.''',
    llm = llm
)

### Tasks

In [ ]:
plan = Task(
    description = "Develop a detailed content outline including introduction, main points, conclusion",
    expected_output = "A detailed outline of content",
    agent = planner
)

write = Task(
    description = "Use the provided content outline to create a engaging blog post on {topic}",
    expected_output = "A detailed blog post",
    agent = writer,
    context = [plan]
)

edit = Task(
    description = "Read the given blog post and make it free of grammatical error and toxic content and provide the blog",
    expected_output = "A detailed blog post",
    agent = editor,
    context = [write]
)


### Crew

In [ ]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose=True
)

In [ ]:
result = crew.kickoff(inputs = {'topic':'Data Science'})

# Agent: Content Planner
## Task: Develop a detailed content outline including introduction, main points, conclusion


# Agent: Content Planner
## Final Answer: 
To create a successful blog post on Data Science, I will follow the following outline:

I. Introduction
    A. Definition of Data Science
    B. Importance of Data Science in today's world
    C. Thesis statement: "Exploring the world of Data Science: Key concepts and applications"

II. Understanding Data Science
    A. History of Data Science
    B. Key components of Data Science
        1. Statistics
        2. Machine Learning
        3. Programming
    C. Role of Data Scientists

III. Applications of Data Science
    A. Business and Marketing
    B. Healthcare
    C. Finance
    D. Education
    E. Other industries

IV. Data Science Process
    A. Data Collection
        1. Sources of data
        2. Types of data
    B. Data Cleaning and Preprocessing
    C. Data Analysis and Visualization
        1


# Agent: Content Wri